In [ ]:
## library import
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import re
from PIL import Image
from glob import glob
import shutil
import xml.etree.ElementTree as et
import random
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [ ]:
## oxford_pet.zip이 보이는지 확인
glob('oxford_pet.zip')

In [ ]:
## 압축풀기
if not os.path.exists('./oxford_pet'):
  !unzip -q oxford_pet.zip -d oxford_pet

In [ ]:
## 압축이 풀린 directory 확인
!ls oxford_pet

In [ ]:
## directory 정보
# os.path.join(): Builds a single path string by intelligently concatenating individual path components
cur_dir = os.getcwd() # 현재 작업 디렉터리를 문자열로 가져온다
data_dir = os.path.join(cur_dir, 'oxford_pet') # 현재 폴더 아래 oxford_pet라는 상위 데이터 폴더 경로를 만든다
image_dir = os.path.join(data_dir, 'images')
bbox_dir = os.path.join(data_dir, 'annotations', 'xmls')  
seg_dir = os.path.join(data_dir, 'annotations', 'trimaps')

In [ ]:
print('data_dir:' ,data_dir)
print('image_dir:' ,image_dir)
print('bbox_dir:', bbox_dir)
print('seg_dir:', seg_dir)

In [ ]:
## image file 수 확인
image_files = [fname for fname in glob(image_dir +  '/*.jpg') if os.path.splitext(fname)[-1] == '.jpg']
print(len(image_files))

In [ ]:
## localization을 위한 annotation이 되어 있는 file의 수 확인
## 위의 이미지 갯수보다 annotation XML 파일 갯수가 적다. annotation XML 파일을 기준으로 해야함.

bbox_files = [fname for fname in glob(bbox_dir +  '/*.xml') if os.path.splitext(fname)[-1] == '.xml']
len(bbox_files)

In [ ]:
## 600 bbox와 매칭되는 이미지를 모우기 위한 새로운 폴더(new_images) 생성

new_image_dir = os.path.join(data_dir, 'new_images')
os.makedirs(new_image_dir, exist_ok=True)

print('new_images:', new_image_dir)

In [ ]:
## BB 정보를 가지고 있는 xml 파일 보기
bbox_files[:10]

In [ ]:
## 600 bbox 리스트 읽어 bbox 이름과 같은 이미지를 new_images 폴더에 복사한다.
# Metadata: Data about data
# It supplies context such as who created a resource, when it was created, and how it should be used or interpreted

for bbox_filename in bbox_files:
  bbox_filename = bbox_filename.split('/')[-1]
  # os.path.splitext(): Separates a path into two parts: the main file name and its extension
  image_name = os.path.splitext(bbox_filename)[0]
  image_file = image_dir + '/' + image_name + '.jpg'
  # print(glob(image_file))
  shutil.copy(image_file, new_image_dir)
  # image_file: The source path-the file you want to duplicate
  # new_iamge_dir: The destination; if it's a directory, the file is copied into that folder with the same file name

In [ ]:
## new_images 폴더에 복사된 이미지 건수를 카운트한다.

new_image_files = glob(new_image_dir + '/*')
len(new_image_files)

In [ ]:
## 600개의 새로 복사된 이미지 리스트

new_image_files[:10]

In [ ]:
## DataFrame 만들기

pets_df = pd.DataFrame(new_image_files)
pets_df

In [ ]:
## 컬럼명 입력

pets_df.columns = ['full_path']
pets_df.head(3)

In [ ]:
## full_path 컬럼에서 이미지 이름을 분리하여 file_name 컬럼명에 저장

pets_df['file_name'] = pets_df['full_path'].str.split('/').str[-1]
pets_df.head(3)

In [ ]:
## file_name 컬럼에서 라벨 분리하여 label 컬럼에 저장
# /d = any digit, + = repeat at least once

pets_df['label'] = pets_df['file_name'].str.replace('_\d+','').str.split('.').str[0]

In [ ]:
## 이미지 패스에서 레이블 분리 및 저장 확인
pets_df.head(10)

In [ ]:
## 이미지 파일명을 입력으로 받아, 같은 이름과 xml 확장자로 변경해서 리턴하는 함수

def name_convert(col):
  bbox_fname = bbox_dir + '/' + col.replace('jpg','xml')
  return bbox_fname

In [ ]:
## name_convert 함수 호출해서 이미지 파일명과 같은 이름의 xml 확장자로 만든다.

pets_df['bbox_full_path'] = pets_df['file_name'].apply(name_convert)
pets_df.head(3)

In [ ]:
## pets_df 데이터 구조 파악

pets_df.info()

In [ ]:
## label 분포 확인
## 각 class마다 200개 이미지 구성

pets_df['label'].value_counts()

In [ ]:
## bbox_full_path 컬럼의 첫번째 데이터 가져오고 해당 내용 보기
# .loc[row_index, column_name]

sample_xml_file = pets_df.loc[0, 'bbox_full_path']
print(sample_xml_file)

# In Jupyter notebooks, placing an exclamation mark(!) before a line- like !ls-
# runs that line as a shell command instead of Python code
!cat {sample_xml_file}

In [ ]:
## 1. xml Annotation 파일 읽어 이지미 크기와 Bounding box 위치을 파악
## 2. xmin, ymin, xmax, ymax 형태를  x, y(중앙), w, h 형태로 변환하여 저장

# ElementTree object stores the XML as a tree of Element nodes, where each node corresponds to a tag(e.g. <size> or <object>)

def xml_annot_getxywh(xmlfile):
  tree = et.parse(xmlfile)

  width = float(tree.find('./size/width').text)
  height = float(tree.find('./size/height').text)
  xmin = float(tree.find('./object/bndbox/xmin').text)
  ymin = float(tree.find('./object/bndbox/ymin').text)
  xmax = float(tree.find('./object/bndbox/xmax').text)
  ymax = float(tree.find('./object/bndbox/ymax').text)
  xc = (xmin + xmax) / 2.
  yc = (ymin + ymax) / 2.
  x = xc / width
  y = yc / height
  w = (xmax - xmin) / width
  h = (ymax - ymin) / height

  return x, y, w, h

In [ ]:
## 샘플 xml 파일에 대한 x, y, w, h 얻어오기

xml_annot_getxywh(sample_xml_file)

In [ ]:
pets_df

In [ ]:
## xml Annot 파일의 x, y, w, h 위치 정보를 가져와 DataFrame에 저장한다.

pets_df['xywh'] = pets_df['bbox_full_path'].apply(xml_annot_getxywh)
pets_df.head()

In [ ]:
## 튜플로 저장된 xywh 컬럼 값들을 하나씩 x, y, w, h 컬럼에 저장한다.

pets_df['x'] = pets_df['xywh'].str[0]
pets_df['y'] = pets_df['xywh'].str[1]
pets_df['w'] = pets_df['xywh'].str[2]
pets_df['h'] = pets_df['xywh'].str[3]

In [ ]:
pets_df

In [ ]:
## label에 대한 라벨인코딩 수행
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
pets_df['label_conv'] = le.fit_transform(pets_df['label'])
le.classes_

In [ ]:
pets_df.head(3)

In [ ]:
## class와 idx 간의 변환

class2idx = { label : idx for idx, label in enumerate(le.classes_) }
idx2class = { idx :label for idx, label in enumerate(le.classes_) }

In [ ]:
print(class2idx)
print(idx2class)

In [ ]:
## 이미지 패스 리스트 만들기

images_list = pets_df['full_path'].values

In [ ]:
## PIL 라이브러리 사용하여 이미지 파일 리스트에서 이미지 패스에 있는 이미지 읽고 리스트에 넣는다.
## 이미지 resize로 맞추지 않으면 뒤쪽에서 Tensor에러 발생한다.

x_image_list = []

for fname in images_list:
  image = Image.open(fname)
  image = image.resize((224,224))
  image = np.array(image)

  x_image_list.append(image)

x_image_list = np.array(x_image_list)

In [ ]:
## 전체 이미지 리스트 형태 보기

x_image_list.shape

In [ ]:
## 이미지 라벨값과 bbox 위치값을 저장한다.

y1_label = pets_df['label_conv'].values
y2_xywh = pets_df[['x', 'y', 'w', 'h']].values

In [ ]:
y1_label[:4]

In [ ]:
y2_xywh[:4]

In [ ]:
# 하이퍼 파라미터 설정

IMG_SIZE = 224
N_BBOX = len(x_image_list)
N_TRAIN = 500
N_VAL = N_BBOX - N_TRAIN

In [ ]:
## Annotation XML 파일 갯수로 리스트 만들고 Shuffle하고 500개 Train set과 100개 Valid set으로 나눈다.

shuffle_list = list(range(N_BBOX))
random.shuffle(shuffle_list)

train_idx_list = shuffle_list[:N_TRAIN]
val_idx_list = shuffle_list[N_TRAIN:]

In [ ]:
## Train set idx 보기
train_idx_list[:10]

In [ ]:
## train 데이터셋 만들기

train_image_list = x_image_list[train_idx_list]
train_y1_label = y1_label[train_idx_list]
train_y2_xywh = y2_xywh[train_idx_list]

In [ ]:
## valid 데이터셋 만들기

valid_image_list = x_image_list[val_idx_list]
valid_y1_label = y1_label[val_idx_list]
valid_y2_xywh = y2_xywh[val_idx_list]

In [ ]:
train_image_list.shape, valid_image_list.shape

In [ ]:
## 이미지와 X/Y/W/H 정보를 가지고 Train / Valid Dataset 만들기

train_dataset_image = tf.data.Dataset.from_tensor_slices((train_image_list, train_y2_xywh))
valid_dataset_image = tf.data.Dataset.from_tensor_slices((valid_image_list, valid_y2_xywh))

In [ ]:
# from_tensor_slices 읽어들인 이미지 리스트와 1개 y값들을 배치, 셔플한다.

# train_dataset = train_dataset_image.batch(16).shuffle(1000).repeat()
# valid_dataset = valid_dataset_image.batch(16).repeat()

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset_image.batch(16).shuffle(1000).cache().prefetch(AUTOTUNE)
valid_dataset = valid_dataset_image.batch(16).cache().prefetch(AUTOTUNE)

In [ ]:
## Hyper Parameters

N_CLASS = len(class2idx)
N_EPOCHS = 40
N_BATCH = 16
IMG_SIZE = 224
learning_rate = 0.0001
steps_per_epoch = N_TRAIN / N_BATCH
validation_steps = int(np.ceil(N_VAL / N_BATCH))

In [ ]:
## valid_dataset에서 1개 가져와 분석해 보자
## 이미지와 라벨로 받고 라벨은 다시 실제 라벨과 x,y,w,h 위치 값으로 나뉜다.

for image, label in valid_dataset.take(1):
  print(image.shape)
  print(label.shape)


In [ ]:
## train dataset에서 1개의 image와 bbox를 읽어서 확인
for image, label in valid_dataset.take(1):

    ''' matplotlib Rectangle 이용하여 사각형 그릴 경우,
    그림을 그리기 위해서 bbox의 왼쪽 아래 (xmin, ymin) 꼭지점 좌표를 계산하고,
    xmin, ymin, w, h 각각을 image size에 맞게 scaling'''

    # x, y(중앙), w, h 형태를 xmin, ymin, w, h 형태로 변환해야 함.

    x = label[:,0]
    y = label[:,1]
    w = label[:,2]
    h = label[:,3]
    xmin = x[0].numpy() - w[0].numpy()/2.
    ymin = y[0].numpy() - h[0].numpy()/2.
    rect_x = int(xmin * IMG_SIZE)
    rect_y = int(ymin * IMG_SIZE)
    rect_w = int(w[0].numpy() * IMG_SIZE)
    rect_h = int(h[0].numpy() * IMG_SIZE)

    ## 그림 그리기
    rect = Rectangle((rect_x, rect_y), rect_w, rect_h, fill=False, color='red')
    plt.axes().add_patch(rect)
    plt.imshow(image[0])
    plt.show()

In [ ]:
# Sequential API를 사용하여 model 구성
def create_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='SAME',
                                  input_shape=(IMG_SIZE, IMG_SIZE, 3)))
    model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1024, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(4, activation='sigmoid'))
    return model

In [ ]:
## Create model, compile & summary
model = create_model()
model.summary()

In [ ]:
def loss_fn(y_true, y_pred):
  return tf.keras.losses.MeanSquaredError()(y_true, y_pred)

In [ ]:
## learning rate scheduing
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate,
                                                          decay_steps=steps_per_epoch*10,
                                                          decay_rate=0.5,
                                                          staircase=True)

# model.compile(optimizer='adam', loss='mse', metrics=['accuracy']) > loss='mse'로 간단하게 할수도 있음.
# 여기서는 직접 loss function을 만들수 있음을 보여줌
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# metric: Additional statistic that Keras calculates after each batch and each epoch to tell you how well the model is doing

In [ ]:
## callbacks : EarlyStopping, ModelCheckpoint
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)
mc = tf.keras.callbacks.ModelCheckpoint('best_model_{val_loss:.2f}.h5', monitor='val_loss', save_best_only=True, verbose=1)

In [ ]:
## Train!
model.fit(train_dataset, steps_per_epoch=steps_per_epoch,
         epochs=N_EPOCHS,
         validation_data=valid_dataset,
         validation_steps=validation_steps,
         callbacks=[es, mc])

### Pretrained MobileNetV2

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Conv2D, ReLU, MaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D

In [ ]:
# include_top: Tells Keras whether to load the network's original classification head
mobilenetv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
mobilenetv2.summary()

In [ ]:
def create_mv_model():
  model = models.Sequential()
  model.add(mobilenetv2)
  model.add(GlobalAveragePooling2D())
  model.add(Dense(4, activation='sigmoid'))
  return model

In [ ]:
## Create model, compile & summary
model = create_mv_model()

## learning rate scheduing
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate,
                                                          decay_steps=steps_per_epoch*5,
                                                          decay_rate=0.5,
                                                          staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
## callbacks : EarlyStopping, ModelCheckpoint
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)
mc = tf.keras.callbacks.ModelCheckpoint('best_model_{val_loss:.2f}.h5', monitor='val_loss', save_best_only=True, verbose=1)

In [ ]:
## Train!
model.fit(train_dataset,
         epochs=N_EPOCHS,
         steps_per_epoch=steps_per_epoch,
         validation_data=valid_dataset,
         validation_steps=validation_steps,
         callbacks=[es, mc])

In [ ]:
## 예측한 bounding box와 ground truth box를 image에 같이 표시
## 정답은 빨간색 box, 예측은 파란색 box

idx = 0
num_imgs = validation_steps
for val_data, val_gt in valid_dataset.take(num_imgs):
    ## 정답 box 그리기 : 빨강박스
    x = val_gt[:,0]
    y = val_gt[:,1]
    w = val_gt[:,2]
    h = val_gt[:,3]
    xmin = x[idx].numpy() - w[idx].numpy()/2.
    ymin = y[idx].numpy() - h[idx].numpy()/2.
    rect_x = int(xmin * IMG_SIZE)
    rect_y = int(ymin * IMG_SIZE)
    rect_w = int(w[idx].numpy() * IMG_SIZE)
    rect_h = int(h[idx].numpy() * IMG_SIZE)

    rect = Rectangle((rect_x, rect_y), rect_w, rect_h, fill=False, color='red')
    plt.axes().add_patch(rect)

    ## 예측 box 그리기 : 파랑박스
    ## validation set에 대해서 bounding box 예측
    prediction = model.predict(val_data)
    pred_x = prediction[:,0]
    pred_y = prediction[:,1]
    pred_w = prediction[:,2]
    pred_h = prediction[:,3]
    pred_xmin = pred_x[idx] - pred_w[idx]/2.
    pred_ymin = pred_y[idx] - pred_h[idx]/2.
    pred_rect_x = int(pred_xmin * IMG_SIZE)
    pred_rect_y = int(pred_ymin * IMG_SIZE)
    pred_rect_w = int(pred_w[idx] * IMG_SIZE)
    pred_rect_h = int(pred_h[idx] * IMG_SIZE)

    pred_rect = Rectangle((pred_rect_x, pred_rect_y), pred_rect_w, pred_rect_h,
                         fill=False, color='blue')
    plt.axes().add_patch(pred_rect)

    ## image와 bbox 함께 출력
    plt.imshow(val_data[idx])
    plt.show()